matching between cars


In [1]:
import numpy as np
from numpy import genfromtxt
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from sklearn.cluster import AgglomerativeClustering
import cv2
from numpy import genfromtxt
import glob 
import copy
from PIL import Image
from iou import iou

In [2]:
zone_dics = np.load('/home/wei/Desktop/test/npy/zone_dics.npy',allow_pickle=True).tolist()
emb_dics = np.load('/home/wei/Desktop/test/npy/emb_dics.npy',allow_pickle=True).tolist()
time_dics = np.load('/home/wei/Desktop/test/npy/time_dics.npy',allow_pickle=True).tolist()

In [4]:
global_id_dic = np.load('/home/wei/Desktop/test/npy/global_id_dic.npy',allow_pickle=True).tolist()

In [5]:
global_id_dic

{1: [(0, 10), (1, 269), (2, 237)],
 2: [(0, 12), (1, 277), (2, 230)],
 3: [(0, 16), (1, 287)],
 4: [(0, 22), (1, 289), (2, 242)],
 5: [(0, 31), (1, 291), (2, 241)],
 6: [(0, 33), (1, 264)],
 7: [(0, 41), (1, 285), (2, 243)],
 8: [(0, 45), (1, 297), (2, 252)],
 9: [(0, 52), (1, 304), (2, 271)],
 10: [(0, 61), (1, 305)],
 11: [(0, 74), (1, 296), (2, 249)],
 12: [(0, 77), (1, 293)],
 13: [(0, 95), (1, 283), (2, 239)],
 14: [(0, 107), (1, 299)],
 15: [(0, 110), (1, 301), (2, 251)],
 16: [(0, 113), (1, 309)],
 17: [(0, 114), (1, 308), (2, 269)],
 18: [(0, 115), (1, 329)],
 19: [(0, 117), (1, 322)],
 20: [(0, 119), (1, 295)],
 21: [(0, 120), (1, 316)],
 22: [(0, 125), (1, 323), (2, 277)],
 23: [(0, 126), (1, 332)],
 24: [(0, 130), (1, 331)],
 25: [(0, 132), (1, 318), (2, 273)],
 26: [(0, 133), (1, 343)],
 27: [(0, 135), (1, 317), (2, 266)],
 28: [(0, 142), (1, 333)],
 29: [(0, 148), (1, 357)],
 30: [(0, 150), (1, 337)],
 31: [(0, 152), (1, 344)],
 32: [(0, 153), (1, 375)],
 33: [(0, 170), (1

In [61]:
print(time_dics[4][85][1],time_dics[5][153][0])
print(distance.cosine(emb_dics[5][141],emb_dics[4][113]))

0.3929299388794034


In [85]:
car_in = []
car_out = []

for car in zone_dics[3]:
    if zone_dics[3][car][1] == 2:
        car_in.append(car)
for car in zone_dics[2]:
    if zone_dics[2][car][0] == 0:
        car_out.append(car)

for car1 in car_in:
    min_dist = 1
    min_car = None
    for car2 in car_out:
        if distance.cosine(emb_dics[3][car1],emb_dics[2][car2])<0.38:
            if time_dics[2][car2][0] - time_dics[3][car1][1] > 400 and time_dics[2][car2][0] - time_dics[3][car1][1] < 800:
                if distance.cosine(emb_dics[3][car1],emb_dics[2][car2])<min_dist:
                    min_car = car2
                    min_dist = distance.cosine(emb_dics[3][car1],emb_dics[2][car2])
    if min_car:
        print(car1,min_car,distance.cosine(emb_dics[3][car1],emb_dics[2][min_car]))

12 127 0.30201562668130955
21 121 0.26436601116418357
22 130 0.3783985671879915
28 129 0.31009302449054443
42 158 0.30599532466148405
43 162 0.27882186362250905
44 160 0.1974272654427064
48 162 0.28439695464688886
60 143 0.22611682234690178
63 163 0.37081427027159863
67 162 0.28575125846829363
75 220 0.33821123157966104
119 255 0.259087086900608
125 268 0.293889205450096
130 313 0.319204897997836
133 286 0.30325501560892043
140 295 0.2999294216445827
144 300 0.35817826176022693
154 309 0.3504945722930227
157 312 0.31773057171493424


In [67]:
seq_in = 2
seq_out = 3
time_thres = (400,800)
emb_thres = 0.38

if seq_in > seq_out: # going right
        car_in = []
        car_out = []
        for car in zone_dics[seq_in]:
            if zone_dics[seq_in][car][1] == 2:
                car_in.append(car)
        for car in zone_dics[seq_out]:
            if zone_dics[seq_out][car][0] == 0:
                car_out.append(car)

        cost = np.ones([len(car_in),len(car_out)])

        for n1,car1 in enumerate(car_in):
            for n2,car2 in enumerate(car_out):
                if distance.cosine(emb_dics[seq_in][car1],emb_dics[seq_out][car2])<0.4:
                    if time_dics[seq_out][car2][0] - time_dics[seq_in][car1][1] > time_thres[0] and time_dics[seq_out][car2][0] - time_dics[seq_in][car1][1] < time_thres[1]:
                        cost[n1][n2] = distance.cosine(emb_dics[seq_in][car1],emb_dics[seq_out][car2])     
                    else:
                        cost[n1][n2] = 1
                else:
                    cost[n1][n2] = 1

        row_ind,col_ind = linear_sum_assignment(cost)

        for match in range(min(len(row_ind),len(col_ind))):
            if cost[row_ind[match]][col_ind[match]]<emb_thres:
                print(car_in[row_ind[match]],car_out[col_ind[match]],cost[row_ind[match]][col_ind[match]])
    
elif seq_in < seq_out: # going left
    car_in = []
    car_out = []
    for car in zone_dics[seq_in]:
        if zone_dics[seq_in][car][1] == 0:
            car_in.append(car)
    for car in zone_dics[seq_out]:
        if zone_dics[seq_out][car][0] == 2:
            car_out.append(car)

    cost = np.ones([len(car_in),len(car_out)])

    for n1,car1 in enumerate(car_in):
        for n2,car2 in enumerate(car_out):
            if distance.cosine(emb_dics[seq_in][car1],emb_dics[seq_out][car2])<0.4:
                if time_dics[seq_out][car2][0] - time_dics[seq_in][car1][1] > time_thres[0] and time_dics[seq_out][car2][0] - time_dics[seq_in][car1][1] < time_thres[1]:
                    cost[n1][n2] = distance.cosine(emb_dics[seq_in][car1],emb_dics[seq_out][car2])     
                else:
                    cost[n1][n2] = 1
            else:
                cost[n1][n2] = 1

    row_ind,col_ind = linear_sum_assignment(cost)

    for match in range(min(len(row_ind),len(col_ind))):
        if cost[row_ind[match]][col_ind[match]]<emb_thres:
            print(car_in[row_ind[match]],car_out[col_ind[match]],cost[row_ind[match]][col_ind[match]])

6 76 0.285925856421933
13 78 0.26012675566924814
18 81 0.23699523596869998
24 99 0.2367131622339822
35 86 0.34225324185494266
38 89 0.37029905961197274
43 83 0.28401840861873606
51 88 0.3366486312173679
69 102 0.27065637423003797
78 100 0.37281152525848693
79 105 0.31972748939868023
77 98 0.16385911563717848
83 93 0.23965959213474208
84 95 0.18352491236051227
92 101 0.21796288538800213
93 94 0.37799476000641175
104 103 0.3403723305806684
105 104 0.21125117018105866
110 113 0.18010785738995516
119 116 0.33028154042492774
166 175 0.18393939864673836
168 171 0.20956251026912331
171 190 0.23334789185487825
204 188 0.358290367060595
214 198 0.31712342436693086
217 205 0.2788952765555831


In [101]:
path_out = '/home/wei/Desktop/track1.txt'
result = np.load('/home/wei/Desktop/test/global_id_dic.npy',allow_pickle=True).tolist()
file = open(path_out,"w")
for car in result:
    print('processing car {}'.format(car))
    for pair in result[car]:
        sct_file = ''
        sct = genfromtxt('/home/wei/Desktop/test/SCT/c0{}.txt'.format(str(41+pair[0])), delimiter=',', dtype=None)
        for label in sct:
            if label[1] == pair[1]:
                file.write(str(41+pair[0])+" ") # cam id
                file.write(str(car)+" ") # car id
                file.write(str(label[0])+" ") # frame id
                file.write(str(int(label[2]))+" "+str(int(label[3]))+" "+str(int(label[4]))+" "+str(int(label[5]))+" "+str(-1)+" "+str(-1)+"\n")

processing car 1
processing car 2
processing car 3
processing car 4
processing car 5
processing car 6
processing car 7
processing car 8
processing car 9
processing car 10
processing car 11
processing car 12
processing car 13
processing car 14
processing car 15
processing car 16
processing car 17
processing car 18
processing car 19
processing car 20
processing car 21
processing car 22
processing car 23
processing car 24
processing car 25
processing car 26
processing car 27
processing car 28
processing car 29
processing car 30
processing car 31
processing car 32
processing car 33
processing car 34
processing car 35
processing car 36
processing car 37
processing car 38
processing car 39
processing car 40
processing car 41
processing car 42
processing car 43
processing car 44
processing car 45
processing car 46
processing car 47
processing car 48
processing car 49
processing car 50
processing car 51
processing car 52
processing car 53
processing car 54
processing car 55
processing car 56
p